# OctoPod - FPL Draft Podcast Analyzer

Run this notebook to fetch, analyze, and summarize FPL podcasts.

**Setup (first time):** Add secrets via 🔑 icon:
- `ANTHROPIC_API_KEY`
- `GCS_CREDENTIALS` (service account JSON)

In [ ]:
# Setup - Run once per session
!git clone https://github.com/CoaseToCoase/OctoPod.git
%cd OctoPod
!pip install -q -e .

In [ ]:
# Load secrets
import os
from google.colab import userdata

os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
os.environ['GCS_CREDENTIALS'] = userdata.get('GCS_CREDENTIALS')
print('✓ Secrets loaded')

In [ ]:
# Sync existing data from GCS (preserves history)
from octopod.gcs import sync_data_from_gcs, is_gcs_configured

if is_gcs_configured():
    sync_data_from_gcs()
    print('✓ Data synced from GCS')
else:
    print('⚠ GCS not configured')

In [ ]:
# Run the pipeline
!octopod run

In [ ]:
# Save all data to GCS
from octopod.gcs import sync_data_to_gcs

if sync_data_to_gcs():
    print('✓ Data saved to GCS')
    print('  gs://octosuitedatahub/octopod/data/videos.json')
    print('  gs://octosuitedatahub/octopod/data/analyses.json')
    print('  gs://octosuitedatahub/octopod/summaries/gw*.json')
else:
    print('⚠ Failed to sync to GCS')